In [ ]:
##Install Java + Spark
!apt-get install -y openjdk-11-jdk
!wget https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar -xzf spark-3.5.0-bin-hadoop3.tgz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libxcomposite1 libxt-dev libxtst6 libxxf86dga1
  openjdk-11-jdk-headless openjdk-11-jre openjdk-11-jre-headless
  session-migration x11-utils
Suggested packages:
  libxt-doc openjdk-11-demo openjdk-11-source visualvm libnss-mdns
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  | fonts-wqy-zenhei fonts-indic mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libxcomposite1 libxt-dev libxtst6 libxxf86dga1
  openjdk-11-jdk openjdk-11-jdk-headless openjdk-

In [ ]:
#Install Dependencies
!pip install clickhouse-connect supabase pyspark pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.0/48.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 57.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 79.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 10.6 MB/s eta 0:00:00


In [ ]:
!pip install clickhouse-connect
!pip install psycopg2-binary

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 41.7 MB/s eta 0:00:00


In [ ]:
!java -version
#to see the version

openjdk version "17.0.17" 2025-10-21
OpenJDK Runtime Environment (build 17.0.17+10-Ubuntu-122.04)
OpenJDK 64-Bit Server VM (build 17.0.17+10-Ubuntu-122.04, mixed mode, sharing)


In [91]:
from supabase import create_client
import clickhouse_connect
from datetime import datetime
import pandas as pd
import os

In [92]:
#Set up Spark Session
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Koinz_ETL") \
    .config("spark.jars.packages", "org.postgresql:postgresql:42.6.0") \
    .getOrCreate()


# ClickHouse Client

In [93]:
#Setup ClickHouse Client
import clickhouse_connect

ch_client = clickhouse_connect.get_client(
    host='cbz3zgbmhe.europe-west4.gcp.clickhouse.cloud',
    user='default',
    password='ojSGSKQMB8xI.',
    secure=True
)


In [94]:
#Fot testing the connection
print("Test ClickHouse:", ch_client.query("SELECT 1").result_set[0][0])

Test ClickHouse: 1


Using updated_at ensures updated rows are captured.

First run → full load from epoch.

In [95]:
##Checkpoint Table
def get_last_checkpoint(job_name: str):
    result = ch_client.query(f"""
        SELECT max(last_processed_timestamp)
        FROM etl_checkpoint
        WHERE job_name = '{job_name}'
    """).result_set

    if not result or result[0][0] is None:
        # First run → full load
        return "1970-01-01 00:00:00.000"

    return result[0][0]

In [96]:
##test
last_ts = get_last_checkpoint("app_user_visits_fact")
print("Last processed timestamp:", last_ts)

Last processed timestamp: 2025-10-01 18:00:46.839000


# Read Data from Postgres

In [110]:
import pandas as pd
from datetime import timedelta
import datetime
import numpy as np

In [98]:
#SetUp supabase for postgres DB
pg_url = "jdbc:postgresql://aws-1-eu-west-1.pooler.supabase.com:5432/postgres"
pg_properties = {
    "user": "postgres.ioluevezsodwdfpxpnce",
    "password": "ONWuOe4nTsKln0ur",
    "driver": "org.postgresql.Driver",
    "sslmode": "require"
}

In [99]:
import pandas as pd
from datetime import timedelta

#Add optional 5-min overlap to catch delayed updates
last_ts_dt = pd.to_datetime(last_ts)
overlap_ts_dt = (last_ts_dt - timedelta(minutes=5))

# Convert datetime object to epoch milliseconds for comparison with BIGINT column
overlap_ts_epoch_ms = int(overlap_ts_dt.timestamp() * 1000)

query = f"""
(
  SELECT *
  FROM app_user_visits_fact
  WHERE updated_at >= {overlap_ts_epoch_ms}
) AS recent_visits
"""

df_postgres = spark.read.jdbc(
    url=pg_url,
    table=query,
    properties=pg_properties
)

print("Rows fetched from Postgres:", df_postgres.count())
df_postgres.show(5, truncate=False)

Rows fetched from Postgres: 4
+------------------------------------+------------+----+-----+------+-------+-----------+---------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+-------------+-------------+-------+----------+------------------------------------+----------+--------+---------------+--------------+
|id                                  |phone_number|seen|state|points|receipt|countryCode|remaining|customer_id                         |branch_id                           |store_id                            |cashier_id                          |created_at   |updated_at   |expired|expires_at|order_id                            |is_deleted|is_fraud|sync_mechanism |is_bulk_points|
+------------------------------------+------------+----+-----+------+-------+-----------+---------+------------------------------------+------------------------------------+-------------------------------

# Transformation & Insert into ClickHouse

In [100]:
#Transform timestamps& Rename -->Matches DateTime64(3)& Preserves millisecond precision &Safe for nulls
from pyspark.sql import functions as F

df_transformed = (
    df_postgres
    # Rename countryCode → country_code
    .withColumnRenamed("countryCode", "country_code")

    # created_at
    .withColumn(
        "created_at",
        (F.col("created_at") / 1000).cast("timestamp")
    )
    # updated_at
    .withColumn(
        "updated_at",
        (F.col("updated_at") / 1000).cast("timestamp")
    )
    # expires_at (nullable)
    .withColumn(
        "expires_at",
        F.when(
            F.col("expires_at").isNotNull(),
            (F.col("expires_at") / 1000).cast("timestamp")
        )
    )
)


In [101]:
#Enforce ClickHouse data types
#1)Flags → UInt8
flag_cols = ["seen", "expired", "is_deleted", "is_fraud"]

for c in flag_cols:
    df_transformed = df_transformed.withColumn(
        c,
        F.col(c).cast("int")
    )


In [102]:
#Enforce ClickHouse data types
#2)Decimal columns
decimal_cols = ["points", "receipt", "remaining"]

for c in decimal_cols:
    df_transformed = df_transformed.withColumn(
        c,
        F.col(c).cast("decimal(10,2)")
    )


In [103]:
COLUMN_MAPPING = {
    "countryCode": "country_code",
    "createdAt": "created_at",
    "updatedAt": "updated_at",
    "expiresAt": "expires_at"
}


In [104]:
for src, tgt in COLUMN_MAPPING.items():
    if src in df_postgres.columns:
        df_transformed = df_transformed.withColumnRenamed(src, tgt)


In [105]:
#Column selection & order
final_cols = [
    "id",
    "phone_number",
    "customer_id",
    "seen",
    "state",
    "expired",
    "is_deleted",
    "is_fraud",
    "points",
    "receipt",
    "remaining",
    "country_code",
    "branch_id",
    "store_id",
    "cashier_id",
    "order_id",
    "created_at",
    "updated_at",
    "expires_at",
    "sync_mechanism",
    "is_bulk_points"
]

df_final = df_transformed.select(*final_cols)
##A Nte we do NOT include _ingested_at ClickHouse fills it automatically

In [106]:
df_final.show(10, truncate=False)


+------------------------------------+------------+------------------------------------+----+-----+-------+----------+--------+------+-------+---------+------------+------------------------------------+------------------------------------+------------------------------------+------------------------------------+-----------------------+-----------------------+----------+---------------+--------------+
|id                                  |phone_number|customer_id                         |seen|state|expired|is_deleted|is_fraud|points|receipt|remaining|country_code|branch_id                           |store_id                            |cashier_id                          |order_id                            |created_at             |updated_at             |expires_at|sync_mechanism |is_bulk_points|
+------------------------------------+------------+------------------------------------+----+-----+-------+----------+--------+------+-------+---------+------------+---------------------------

In [107]:
expected_cols = set(final_cols)
actual_cols = set(df_final.columns)

missing = expected_cols - actual_cols
extra = actual_cols - expected_cols

if missing:
    raise Exception(f"Missing columns before ClickHouse load: {missing}")

if extra:
    print(f"Warning: extra columns ignored: {extra}")


In [111]:
CLICKHOUSE_COLUMNS = [
    "id",
    "phone_number",
    "customer_id",
    "seen",
    "state",
    "expired",
    "is_deleted",
    "is_fraud",
    "points",
    "receipt",
    "remaining",
    "country_code",
    "branch_id",
    "store_id",
    "cashier_id",
    "order_id",
    "created_at",
    "updated_at",
    "expires_at",
    "sync_mechanism",
    "is_bulk_points"
]

DATETIME_COLS = ["created_at", "updated_at", "expires_at"]
UINT8_COLS = ["seen", "expired", "is_deleted", "is_fraud"]


def clean_datetime(series):
    def _convert(x):
        if pd.isna(x):
            return None
        if isinstance(x, pd.Timestamp):
            return x.to_pydatetime()
        if isinstance(x, datetime.datetime):
            return x
        return None
    return series.apply(_convert)


def normalize_uint8(series, default=0):
    def _convert(x):
        if x is None or pd.isna(x):
            return default
        return int(bool(x))
    return series.apply(_convert)


def insert_to_clickhouse(df_spark, job_name="app_user_visits_fact"):
    if df_spark.rdd.isEmpty():
        print("No new data to insert.")
        return

    # Convert to Pandas
    df = df_spark.toPandas()

    # Ensure required columns exist
    missing_cols = set(CLICKHOUSE_COLUMNS) - set(df.columns)
    if missing_cols:
        raise ValueError(f"Missing required columns: {missing_cols}")

    # Clean datetime columns
    for col in DATETIME_COLS:
        df[col] = clean_datetime(df[col])

    # Normalize UInt8 flags
    for col in UINT8_COLS:
        df[col] = normalize_uint8(df[col])

    # Replace NaN → None
    df = df.replace({np.nan: None})

    # Insert into fact table
    ch_client.insert(
        table="app_user_visits_fact",
        column_names=CLICKHOUSE_COLUMNS,
        data=df[CLICKHOUSE_COLUMNS].values.tolist()
    )

    # Compute checkpoint safely
    updated_ts_series = df["updated_at"].dropna()
    if updated_ts_series.empty:
        raise ValueError("All updated_at values are NULL. Cannot update checkpoint.")

    max_ts = updated_ts_series.max()

    # Insert checkpoint
    ch_client.insert(
        table="etl_checkpoint",
        column_names=[
            "job_name",
            "last_processed_timestamp",
            "rows_processed",
            "status",
            "error_message",
            "updated_at"
        ],
        data=[[
            job_name,
            max_ts,
            int(len(df)),
            "success",
            None,
            datetime.datetime.now()
        ]]
    )

    print(f"Inserted {len(df)} rows. Checkpoint updated to {max_ts}")


In [112]:
insert_to_clickhouse(df_final)

Inserted 4 rows. Checkpoint updated to 2025-10-01 18:00:46.839000
